In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
from ipywidgets import interact, ToggleButtons, SelectMultiple
import copy

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

In [4]:
oakdf = pd.read_excel('../data/plant_cycle_time/oakcreek_cycle_time_0119_0520.xlsx', header=1)

# For Pittsburg Meeting
## Consolidate products/reactors

Volumes by family – delete what I don’t make more than 1-2 times per year. See how much capacity is opened up – shutdown reactors 7, 2, 3. Keep just higher volume products.

In [5]:
# select oakdf or cledf
df = copy.deepcopy(oakdf)

In [6]:
dates = [i for i in df.columns if ('Date' in i or 'First' in i or 'Last' in i)]
times = [i for i in df.columns if 'Time' in i]
other = [i for i in df.columns if (i not in dates and i not in times)]
[print(i) for i in dates]
print()
[print(i) for i in times]
print()
[print(i) for i in other]

Batch Completion Date
First Inv Pick
First Reservation Created
Last Inv Pick
Last Reservation Created
First Formulated Consumed Material
Last Formulated Consumed Material
First QC Consumed Material
Last QC Consumed Material
First Material Consumption to First Sample
TO.80 Log Date
Preship And Fill Date
TO.80 Approval Date
Min SKU WIP Start Date
First WIP Completion Transaction
Last WIP Completion Transaction
TO.90 Log Date
TO.90 Approval Date
Min Date
Max Date

Picking Time
Total PA Time
Total Formulated Material Consumption Time
QC Adjustment Material Consumption Time
Total Material Consumption Time
T80 TO PSF Time
Total TO.80 Time
Container Filling Time
Total Fill Time
Total TO.90 Time
Total Cycle Time

Inventory Org Code
Batch Number
Cost Center
Technology
Recipe No
Recipe Version
Product
Inventory Category
Equalization Lot Number
Parent Batch Planned Qty
Parent Batch Actual Qty
Parent Batch UOM
Move Order Created


[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [7]:
for col in times:
    try:
        df[col] = pd.to_timedelta(df[col])
        df[col] = df[col].dt.total_seconds()/60/60 #convert timedelta to hours
    except:
        print(col)

Total Material Consumption Time


In [8]:
dff = df[['Product', 'Technology', 'Total Cycle Time', 'Parent Batch Actual Qty', 'Batch Completion Date']]
dff.shape

(21410, 5)

In [9]:
dff.head()

,Product,Technology,Total Cycle Time,Parent Batch Actual Qty,Batch Completion Date
0,CMPP4100H,A-LIQUID,106.394444,3253.00000,2019-02-23
1,SPM76570,I-LIQUID,1514.857778,514.75614,2019-05-13
2,SPM76570,I-LIQUID,3.814722,256.60143,2019-06-03
3,WOSPM76570/TT,I-LIQUID,3.814722,0.00000,2019-06-03
4,W5646-6,I-LIQUID,109.316944,268.77470,2019-02-23


In [10]:
dff = dff.loc[dff['Total Cycle Time'] > .25] # greater than a quarter hour
dff = dff.loc[dff['Parent Batch Actual Qty'] > 10] # more than 10 gal
dff = dff.loc[dff['Technology'] != 'RESINS']
dff = dff.loc[dff['Technology'] != 'RAWMATERIAL']
dff = dff.loc[dff['Technology'] != 'Unspecified']
dff = dff.loc[dff['Technology'] != 'A-Solvent resin']
dff = dff.loc[dff['Technology'] != 'I-E-COAT']
dff = dff.loc[dff['Technology'] != 'A-E-COAT']
dff = dff.loc[dff['Technology'] != 'Ecoat']
dff = dff.dropna()
dff.shape

(19895, 5)

In [11]:
dff['Family'] = dff['Product'].str.replace('\d+', '')
dff['Gal/Hr'] = dff['Parent Batch Actual Qty']/dff['Total Cycle Time']

dff['Product Total Batches'] = dff.groupby(['Product'])['Gal/Hr'].transform('count')
dff['Product Total Batches Bin'] = pd.cut(dff['Product Total Batches'], [1,10,20,50,400])
dff['Family Total Batches'] = dff.groupby(['Family'])['Gal/Hr'].transform('count')

In [12]:
def plot_one(x=dff.columns[2:][::-1], y=dff.columns[2:], c=dff.columns[1:]):
    fig = px.scatter(dff, x=x, y=y, color=c,
                    title="{} vs {}".format(y,x))
    fig.show()

In [13]:
df['Cost Center'].unique()

array(['S', 'O', 'R', '8', 'N', 'I', 'A', 'T', '1', 'U', '7', 'P', nan],
      dtype=object)

In [14]:
dff.groupby('Product Total Batches Bin')['Product'].nunique()

Product Total Batches Bin
(1, 10]      1625
(10, 20]      412
(20, 50]      120
(50, 400]      14
Name: Product, dtype: int64

In [15]:
interact(plot_one)

In /home/wab665/anaconda3/envs/py37/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/wab665/anaconda3/envs/py37/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/wab665/anaconda3/envs/py37/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/wab665/anaconda3/envs/py37/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/wab665/anac

interactive(children=(Dropdown(description='x', options=('Family Total Batches', 'Product Total Batches Bin', …

<function __main__.plot_one(x=Index(['Family Total Batches', 'Product Total Batches Bin',
       'Product Total Batches', 'Gal/Hr', 'Family', 'Batch Completion Date',
       'Parent Batch Actual Qty', 'Total Cycle Time'],
      dtype='object'), y=Index(['Total Cycle Time', 'Parent Batch Actual Qty', 'Batch Completion Date',
       'Family', 'Gal/Hr', 'Product Total Batches',
       'Product Total Batches Bin', 'Family Total Batches'],
      dtype='object'), c=Index(['Technology', 'Total Cycle Time', 'Parent Batch Actual Qty',
       'Batch Completion Date', 'Family', 'Gal/Hr', 'Product Total Batches',
       'Product Total Batches Bin', 'Family Total Batches'],
      dtype='object'))>

In [16]:
computation = ToggleButtons(
    options=['Average', 'Median', 'Sum'],
    description='Computation:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Take the Average', 'Take the Median', 'Take the Sum']
)
def plot_two(computation=computation, 
             metric=dff.select_dtypes(np.number).columns.tolist()):
    if computation == 'Average':
        dfg = pd.DataFrame(dff.groupby('Product Total Batches Bin')[metric].mean())
    elif computation == 'Median':
        dfg = pd.DataFrame(dff.groupby('Product Total Batches Bin')[metric].median())
    else:
        dfg = pd.DataFrame(dff.groupby('Product Total Batches Bin')[metric].sum())
    dfg = dfg.reset_index()
    fig = px.bar(x=dfg['Product Total Batches Bin'].astype(str), y=dfg[metric], 
                 title="{} {} as a Function of Times Product Was Made".format(computation, metric))
    fig.show()

In [17]:
interact(plot_two)

interactive(children=(ToggleButtons(description='Computation:', options=('Average', 'Median', 'Sum'), tooltips…

<function __main__.plot_two(computation=ToggleButtons(description='Computation:', options=('Average', 'Median', 'Sum'), tooltips=('Take the Average', 'Take the Median', 'Take the Sum'), value='Average'), metric=['Total Cycle Time', 'Parent Batch Actual Qty', 'Gal/Hr', 'Product Total Batches', 'Family Total Batches'])>

In [18]:
selected = SelectMultiple(
    options=[pd._libs.interval.Interval(1, 10, closed='right'),
             pd._libs.interval.Interval(10, 20, closed='right'),
             pd._libs.interval.Interval(20, 50, closed='right'),
             pd._libs.interval.Interval(50, 400, closed='right')],
    value=[pd._libs.interval.Interval(1, 10, closed='right'),
             pd._libs.interval.Interval(10, 20, closed='right')],
    description='Bins',
    disabled=False
)
computation = ToggleButtons(
    options=['Average', 'Median'],
    description='Computation:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Take the Average', 'Take the Median']
)
def plot_three(selected=selected, computation=computation):
    available_cycle_hours = dff.loc[dff['Product Total Batches Bin']
                                    .isin(selected)]['Total Cycle Time'].sum()
    if computation == 'Average':
        remaining_avg_rate = dff.loc[~dff['Product Total Batches Bin']
                                         .isin(selected)]['Gal/Hr'].mean()
        prev_avg_rate = dff['Gal/Hr'].mean()
    else:
        remaining_avg_rate = dff.loc[~dff['Product Total Batches Bin']
                                         .isin(selected)]['Gal/Hr'].median()
        prev_avg_rate = dff['Gal/Hr'].median()
    new_production_gal = available_cycle_hours * remaining_avg_rate

    
    prev_production = dff['Parent Batch Actual Qty'].sum()
    dfff = pd.DataFrame([[prev_avg_rate, remaining_avg_rate],
                  [prev_production,prev_production+new_production_gal]],
                columns=['Previous', 'New'],
                index=['Rate (Gal/Hr)', 'Production (Gal)']).T
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=['Rate (Gal/Hr)', 'Production (Gal)'])
    
    fig.add_trace(
        px.bar(dfff, y='Rate (Gal/Hr)',  x=dfff.index).data[0],
        row=1, col=1,
    )
    
    fig.add_trace(
        px.bar(dfff, y='Production (Gal)',  x=dfff.index).data[0],
        row=1, col=2,
    )
    fig.update_layout(
        title_text='Impact of Repurpoposing Capacity to More Frequently Made Products')
    fig.show()

In [19]:
35/20

1.75

In [20]:
interact(plot_three)

interactive(children=(SelectMultiple(description='Bins', index=(0, 1), options=(Interval(1, 10, closed='right'…

<function __main__.plot_three(selected=SelectMultiple(description='Bins', index=(0, 1), options=(Interval(1, 10, closed='right'), Interval(10, 20, closed='right'), Interval(20, 50, closed='right'), Interval(50, 400, closed='right')), value=(Interval(1, 10, closed='right'), Interval(10, 20, closed='right'))), computation=ToggleButtons(description='Computation:', options=('Average', 'Median'), tooltips=('Take the Average', 'Take the Median'), value='Average'))>

# Cycle Times